In [1]:
vet inte vad jag gör :)

SyntaxError: invalid syntax (<ipython-input-1-58df5c858636>, line 1)

In [1]:
lines = []
with open('./data/train.csv', 'r') as file:
    for i,line in enumerate(file):
        if i < 1500:
            lines.append(line)
        else:
            break
with open('./data/short_train.csv', 'w') as file:
    for line in lines:
        file.write(line)

